In [1]:
import requests
import re
from bs4 import BeautifulSoup
import time
import random
from ebooklib import epub #http://docs.sourcefabric.org/projects/ebooklib/en/latest/tutorial.html#creating-epub

In [2]:
class Page:
    def __init__(self, url, base_url):
        self.url = url
        self.base_url = base_url
        req = requests.get(self.url, headers)
        self.soup = BeautifulSoup(req.content)
        
    def get_next_url(self):
        next_url = self.soup.find('a', string=re.compile(css_nexttext))['href']
        next_url = base_url + next_url
        return next_url

    def get_booktitle(self):
        """soup.find('span', class_='shuming')"""
        title = self.soup.find(**css_booktitle).text
        title = self.clean_text(title)
        return title

    def get_author(self):
        """soup.find('h1', id='timu')"""
        author = self.soup.find(**css_author).text
        author = self.clean_text(author)
        return author
    
    def get_chaptitle(self):
        """soup.find('h1', id='timu')"""
        title = self.soup.find(**css_chaptitle).text
        title = self.clean_text(title)
        return title
    
    def get_content(self):
        """soup.find('h1', id='timu')"""
        # find all 'p'
        # content = self.soup.find_all(**css_content)
        # content = [x.get_text() for x in content]
        # content = '\n'.join(content)
        content = self.soup.find('div', class_=re.compile('cont')).get_text('\n')
#         # print(page.soup.find('div', class_=re.compile('cont')).get_text("\n"))
        # clean
        content = content.strip()
        content = content.replace('\u3000', '\n')
        content = content.replace('\xa0', '\n')
        content = content.replace('<br/>', '\n')
        
        content = re.sub(r'\n+', '\n', content)
        content = '\t'.join(('\n'+content.lstrip()).splitlines(keepends=True))
        
        # add chaptitle
        chaptitle = self.get_chaptitle()
        print(chaptitle, len(content))
        content = chaptitle + '\n' + content
        
        # html
        content = content.replace('\n', '<br>') #.sub(r'\n', '<br>', content) #
        content = content.replace('\t', '&emsp;') #.sub(r'\t', '&emsp;', content) #
        return content
        
    def clean_text(self, text):
        text = text.replace('小說', '')
        text = text.replace('作者', '')
        text = text.replace('：', ' ')
        text = text.replace(':', ' ')
        text = re.sub(r' +', ' ', text)
        text = text.strip()
        return text
    
def run_test():
    page = Page(url, base_url)
    print('Booktitle:\t', page.get_booktitle())
    print('Author:\t\t', page.get_author())
    print('chap:\t', page.get_chaptitle())
    print('Next url:\t', page.get_next_url())
    print('Content length:\t', len(page.get_content()))
    # print(page.get_content())
    
def create_book(first_page, language='zh'):
    booktitle = first_page.get_booktitle()
    print('Booktitle:', booktitle)

    # define book
    book = epub.EpubBook()
    book.set_title(booktitle)
    book.set_language(language)
    try:
        author = first_page.get_author()
        print('Author:', author)
        book.add_author(author)
    except:
        print('No author added')
    return book

def create_chap(page):
    chaptitle = page.get_chaptitle()
    chap = epub.EpubHtml(title=chaptitle, file_name=chaptitle + '.xhtml')
    chap.set_content(page.get_content())
    # chap.content = content
    return chap

def write_book(book, booktitle, toc, spine):
    # toc
    toc = tuple(toc)
    book.toc = toc

    # add default NCX and Nav file
    book.add_item(epub.EpubNcx())
    book.add_item(epub.EpubNav())

    # spine = tuple(spine)
    book.spine = ['nav', *spine]

    # write to the file
    epub.write_epub('book/' + booktitle + '.epub', book, {})

# Config

In [3]:
count = 1
url = """
https://tw.uukanshu.com/b/150317/50458.html
"""
url = url.strip()
base_url = 'https://tw.uukanshu.com'

# css
css_booktitle    = {'name': 'span', 'class_': 'shuming'}
css_author       = {'name': 'span', 'class_': 'author'}
css_chaptitle = {'name': 'h1', 'id': 'timu'}
css_content      = {'name': 'p'}
css_nexttext    = '下一章'

# optional
language = 'zh'
file_format = "epub"

headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }

run_test()

Booktitle:	 簽到千年我怎么成人族隱藏老祖了
Author:		 超喜歡吃辣椒
chap:	 第263章 天青山脈
Next url:	 https://tw.uukanshu.com/b/150317/50542.html
第263章 天青山脈 2473
Content length:	 2787


# Save book

In [4]:
first_page = Page(url, base_url)
booktitle = first_page.get_booktitle()
continue_extract = True
counter = 0

"""
declare file format
"""
if file_format == 'epub':
    book = create_book(first_page, language=language)
    # define toc
    toc = []
    spine = []
else:
    filename = booktitle + ".txt"
    f = open(filename, "a+")

"""
start scraping
"""
while url is not None and continue_extract and (counter < count or count == 0):
    counter += 1
    time.sleep(random.randint(3, 6))
    
    page = Page(url, base_url)
    
    # save as epub or txt file
    if file_format == 'epub':
        chap = create_chap(page)

        # add chap
        book.add_item(chap)
        spine.append(chap)
        # create toc
        link = epub.Link(chap.file_name, chap.title, chap.id)
        toc.append(link)
    else:
        f.write(page.get_content())

    # get next url
    try:
        next_url = page.get_next_url()
        url = next_url
    except:
        print("End of url")
        break
        
"""
save file
"""
if file_format == 'epub':
    write_book(book, booktitle, toc, spine)
else:
    f.close()
    
print("=====\nDONE")

Booktitle: 簽到千年我怎么成人族隱藏老祖了
Author: 超喜歡吃辣椒
第263章 天青山脈 2473
=====
DONE


In [5]:
# assert False, "breakpoint"

# Test

In [6]:
page = Page(url, base_url)
print(page.get_content())
# page.get_content()

第264 荒界之靈 2582
<br>          如果喜歡《簽到千年我怎么成人族隱藏老祖了》，請把網址發給您的朋友。<br>  收藏本頁請按<br>  Ctrl<br>  +<br>  D，為方便下次閱讀也可把本書<br>  添加到桌麵<br>  ，添加桌麵請猛擊<br>  這裏<br>  。r>  那些蠻域境內的人，只是看到太陽，還有天空有點不一樣，其它的還并未有什么變化。<br>  但在蠻域邊境的人族強者。<br>  才真真切切的感受到了不同。<br>  他們來到蠻域邊緣。<br>  看著那斷裂的楚橋。<br>  洶涌的海浪已經不見，入目所及皆是高山。<br>  “這好像是換了一個世界一樣！”<br>  戰斧走出蠻域的范圍，來到外面，深吸了一口氣，有了很不一樣的感覺。<br>  “到底發生了什么事？”<br>  “我族前輩，與那異族恐怖存在對戰，最后的結果又是怎么樣的？”<br>  他目光看向遠處。<br>  …………<br>  “小楚哥哥！”<br>  趙玉靈林雪靈騎著馬直接闖了進來。<br>  后面跟著林騰一家。<br>  看著悠閑的在椅子上搖晃的楚河，她們露出一臉驚喜的表情。<br>  “怎么了！”<br>  楚河看著她們幾個，眼睛紅紅的，不由露出詫異。<br>  “沒，沒什么！”<br>  趙玉靈搖頭！<br>  然后堅定的道，：“小楚哥哥，我要認真修煉，要跟你一樣厲害！”<br>  “你一直很努力的啊！”<br>  “我也沒看你偷懶！”<br>  楚河笑著道。<br>  趙玉靈聞言，神情一怔，眼珠一轉。<br>  不由點點頭。<br>  好像確實如此，她一般修煉都沒怎么偷懶來著。<br>  之所以還是實力不高！<br>  說到底還是天賦不行！<br>  想到此，趙玉靈一臉的失落，感覺挫敗不已。<br>  林雪靈同樣如此，這些年她雖然修煉起來，激情不如從前，但其實還算努力。<br>  卻依舊無法幫到楚河一丁點！<br>  這是年齡差距所帶來的！<br>  “你們怎么了？”<br>  楚河看她們一副受打擊的模樣，感覺疑惑。<br>  “沒什么呢！”<br>  趙玉靈搖頭。<br>  然后又露出笑容道，：“小楚哥哥，好久沒聽你講故事了！再給我們講講吧！”<br>  …………<br>  人族來到了陌生的地界。<br>  經歷了初期的不適。<br>  但

In [7]:
print(page.soup.find('div', class_=re.compile('cont')).get_text("\n"))
page.soup.find('div', class_=re.compile('cont')).get_text("\n")















隨著蠻域從空間隧道之中出來。
然后又接連發生震動。
蠻域之中的人，開始從沉睡之中接連醒來！
發現，一切的不適都已經結束。
看著天上的三個太陽，他們總感覺，一場激戰之后，世界變的好像不一樣了！
那些蠻域境內的人，只是看到太陽，還有天空有點不一樣，其它的還并未有什么變化。
但在蠻域邊境的人族強者。
才真真切切的感受到了不同。
他們來到蠻域邊緣。
看著那斷裂的楚橋。
洶涌的海浪已經不見，入目所及皆是高山。
“這好像是換了一個世界一樣！”
戰斧走出蠻域的范圍，來到外面，深吸了一口氣，有了很不一樣的感覺。
“到底發生了什么事？”
“我族前輩，與那異族恐怖存在對戰，最后的結果又是怎么樣的？”
他目光看向遠處。
…………
“小楚哥哥！”
趙玉靈林雪靈騎著馬直接闖了進來。
后面跟著林騰一家。
看著悠閑的在椅子上搖晃的楚河，她們露出一臉驚喜的表情。
“怎么了！”
楚河看著她們幾個，眼睛紅紅的，不由露出詫異。
“沒，沒什么！”
趙玉靈搖頭！
然后堅定的道，：“小楚哥哥，我要認真修煉，要跟你一樣厲害！”
“你一直很努力的啊！”
“我也沒看你偷懶！”
楚河笑著道。
趙玉靈聞言，神情一怔，眼珠一轉。
不由點點頭。
好像確實如此，她一般修煉都沒怎么偷懶來著。
之所以還是實力不高！
說到底還是天賦不行！
想到此，趙玉靈一臉的失落，感覺挫敗不已。
林雪靈同樣如此，這些年她雖然修煉起來，激情不如從前，但其實還算努力。
卻依舊無法幫到楚河一丁點！
這是年齡差距所帶來的！
“你們怎么了？”
楚河看她們一副受打擊的模樣，感覺疑惑。
“沒什么呢！”
趙玉靈搖頭。
然后又露出笑容道，：“小楚哥哥，好久沒聽你講故事了！再給我們講講吧！”
…………
人族來到了陌生的地界。
經歷了初期的不適。
但也很快調整了過來。
日子照常過，修行也不能落下。
不過，他們很快發現了一個問題。
他們在蠻域之中修行是正常的！
但出了蠻域，即便外面的環境，比以前的東蒼域好了不知多少，隨便一個好一點的地方，都如同洞天福地。
但它們修行之時，卻會感覺到阻澀感，很不順利。
還很難靜下心來。
修煉的時候，會感覺腦中一片亂糟糟的！
這片天地，似乎不喜歡，在排斥它們。
不過，還好，在蠻域他們還能照常修煉。
外面得到的寶物，也能夠使用，效果并不差。
否則直接完蛋！
楚河來到蠻域之外，修行感受了一下。
卻發現，他并沒有遇到其

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n隨著蠻域從空間隧道之中出來。\n然后又接連發生震動。\n蠻域之中的人，開始從沉睡之中接連醒來！\n發現，一切的不適都已經結束。\n看著天上的三個太陽，他們總感覺，一場激戰之后，世界變的好像不一樣了！\n那些蠻域境內的人，只是看到太陽，還有天空有點不一樣，其它的還并未有什么變化。\n但在蠻域邊境的人族強者。\n才真真切切的感受到了不同。\n他們來到蠻域邊緣。\n看著那斷裂的楚橋。\n洶涌的海浪已經不見，入目所及皆是高山。\n“這好像是換了一個世界一樣！”\n戰斧走出蠻域的范圍，來到外面，深吸了一口氣，有了很不一樣的感覺。\n“到底發生了什么事？”\n“我族前輩，與那異族恐怖存在對戰，最后的結果又是怎么樣的？”\n他目光看向遠處。\n…………\n“小楚哥哥！”\n趙玉靈林雪靈騎著馬直接闖了進來。\n后面跟著林騰一家。\n看著悠閑的在椅子上搖晃的楚河，她們露出一臉驚喜的表情。\n“怎么了！”\n楚河看著她們幾個，眼睛紅紅的，不由露出詫異。\n“沒，沒什么！”\n趙玉靈搖頭！\n然后堅定的道，：“小楚哥哥，我要認真修煉，要跟你一樣厲害！”\n“你一直很努力的啊！”\n“我也沒看你偷懶！”\n楚河笑著道。\n趙玉靈聞言，神情一怔，眼珠一轉。\n不由點點頭。\n好像確實如此，她一般修煉都沒怎么偷懶來著。\n之所以還是實力不高！\n說到底還是天賦不行！\n想到此，趙玉靈一臉的失落，感覺挫敗不已。\n林雪靈同樣如此，這些年她雖然修煉起來，激情不如從前，但其實還算努力。\n卻依舊無法幫到楚河一丁點！\n這是年齡差距所帶來的！\n“你們怎么了？”\n楚河看她們一副受打擊的模樣，感覺疑惑。\n“沒什么呢！”\n趙玉靈搖頭。\n然后又露出笑容道，：“小楚哥哥，好久沒聽你講故事了！再給我們講講吧！”\n…………\n人族來到了陌生的地界。\n經歷了初期的不適。\n但也很快調整了過來。\n日子照常過，修行也不能落下。\n不過，他們很快發現了一個問題。\n他們在蠻域之中修行是正常的！\n但出了蠻域，即便外面的環境，比以前的東蒼域好了不知多少，隨便一個好一點的地方，都如同洞天福地。\n但它們修行之時，卻會感覺到阻澀感，很不順利。\n還很難靜下心來。\n修煉的時候，會感覺腦中一片亂糟糟的！\n這片天地，似乎不喜歡，在排斥它們。\n不過，還好，在